In [13]:
import os
import time

import aprofiles as apro
import numpy as np
import xarray as xr

start_time = time.time()

BASE_DIR_OUT = 'data/v-profiles'
date = '2021-09-09'
yyyy = date.split('-')[0]
mm = date.split('-')[1]
dd = date.split('-')[2]

In [4]:
datepath = os.path.join(BASE_DIR_OUT,yyyy,mm,dd)
onlyfiles = [f for f in os.listdir(datepath) if os.path.isfile(os.path.join(datepath, f))]

In [5]:
filename = onlyfiles[0]
ds = xr.open_dataset(os.path.join(datepath, filename), decode_times=True)

In [20]:
def get_lowest_clouds(ds):
    # returns an array of the altitude (in m, ASL) of the lowest cloud for each timestamp

    def _get_true_indexes(mask):
        # mask: list of Bool
        # returns a list indexes where the mask is True
        return [i for i, x in enumerate(mask) if x]

    lowest_clouds = []
    for i in np.arange(len(ds.time.data)):
        i_clouds = _get_true_indexes(ds.clouds_bases.data[i, :])
        if len(i_clouds) > 0:
            lowest_clouds.append(ds.altitude.data[i_clouds[0]])
        else:
            lowest_clouds.append(np.nan)

    return np.asarray(lowest_clouds)

In [24]:
lowest_clouds = get_lowest_clouds(ds)
low_level_mask = lowest_clouds<=6000

In [34]:
scene = []
for i, lowest_cloud in enumerate(lowest_clouds):
    if lowest_cloud<=6000:
        scene.append('cloud<6km')
    elif lowest_cloud>6000:
        scene.append('cloud>6km')
    else:
        scene.append('aer')
    # overwrite result based on foc
    if ds.foc.data[i]:
        scene[i] = 'foc'

In [36]:
ds.foc.data

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,